# "저공해차 보급촉진을 위한 제도운영 지원 및 개선방안 마련" 데이터 분석

## library

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### data load

In [8]:
data = pd.read_excel('raw/20년/2020년06월.xlsx')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204171 entries, 0 to 204170
Data columns (total 11 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   충전소명       204171 non-null  object 
 1   충전기ID      204171 non-null  int64  
 2   지역         204171 non-null  object 
 3   시군구        204171 non-null  object 
 4   주소         204171 non-null  object 
 5   충전기용량(kW)  204171 non-null  int64  
 6   충전기타입      204171 non-null  object 
 7   충전시작일시     204171 non-null  int64  
 8   충전종료일시     204171 non-null  int64  
 9   충전시간       204171 non-null  object 
 10  충전량        204171 non-null  float64
dtypes: float64(1), int64(4), object(6)
memory usage: 17.1+ MB


In [235]:
total_data = pd.DataFrame()
# 2020년
count = 0
for m in range(1, 13):
    y = 20
    s = f'raw/{y}년/20{y}년{m:0>2}월.xlsx'
    m_data = pd.read_excel(s)
    count += m_data.shape[0]
    total_data = pd.concat([total_data, m_data], ignore_index=True)
# 2021년
for m in range(1, 13):
    y = 21
    s = f'raw/{y}년/{y}년{m:0>2}월.xlsx'
    m_data = pd.read_excel(s)
    count += m_data.shape[0]
    total_data = pd.concat([total_data, m_data], ignore_index=True)
# 2022년
for m in range(1, 9):
    y = 22
    s = f'raw/{y}년/{y}년{m:0>2}월.xlsx'
    m_data = pd.read_excel(s)
    count += m_data.shape[0]
    total_data = pd.concat([total_data, m_data], ignore_index=True)

In [ ]:
print(f'개별 데이터 개수 합산 : {count}')
print(f'총 데이터 개수 : {total_data.shape[0]}')

In [78]:
total_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11665822 entries, 0 to 11665821
Data columns (total 12 columns):
 #   Column     Dtype  
---  ------     -----  
 0   충전소명       object 
 1   충전기ID      int64  
 2   지역         object 
 3   시군구        object 
 4   주소         object 
 5   충전기용량(kW)  float64
 6   충전기타입      object 
 7   충전시작일시     int64  
 8   충전종료일시     int64  
 9   충전시간       object 
 10  충전량        float64
 11  충전용량       object 
dtypes: float64(2), int64(3), object(7)
memory usage: 1.0+ GB


In [80]:
total_data.to_csv('2020-22년_급속충전.csv', encoding='utf-8')

In [81]:
df = total_data.copy()

## 기본 분석
- 충전소명 : O
- 충전기ID : int64
- 지역 : O
- 시군구 : O
- 주소 : O
- 충전기용량(kW) : float64
- 충전기타입 : object 
- 충전시작일시 : int64  
- 충전종료일시 : int64  
- 충전시간 : object 
- 충전량 : float64
- 충전용량 : object 

#### 특이사항
- 2020년 12월까지 충전기용량(kW)가 2021년 1월부터 충전용량으로 변경
    - 2020년 12월까지 데이터의 충전기용량(kW) 값을 충전용량으로 이동

In [86]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11665822 entries, 0 to 11665821
Data columns (total 12 columns):
 #   Column     Dtype  
---  ------     -----  
 0   충전소명       object 
 1   충전기ID      int64  
 2   지역         object 
 3   시군구        object 
 4   주소         object 
 5   충전기용량(kW)  float64
 6   충전기타입      object 
 7   충전시작일시     int64  
 8   충전종료일시     int64  
 9   충전시간       object 
 10  충전량        float64
 11  충전용량       object 
dtypes: float64(2), int64(3), object(7)
memory usage: 1.0+ GB


In [82]:
df.head()

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,충전량,충전용량
0,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200117193331,20200117201417,00:40:00,25.47,NaN
1,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200130143545,20200130150645,00:30:31,17.57,NaN
2,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200105005728,20200105011329,00:15:35,6.98,NaN
3,내린천휴게소 양양방향,3,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),100.0,DC콤보,20200120120215,20200120124708,00:41:00,15.30,NaN
4,한국광기술원,1,광주광역시,북구,광주광역시 북구 첨단 벤처로 108번길 9,50.0,DC차데모+AC3상+DC콤보,20200123085417,20200123092554,00:31:14,21.92,NaN


In [89]:
df[df['충전용량'].notnull() == True].head()

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,충전량,충전용량
2796614,군위휴게소(부산방향),1,경상북도,군위군,경상북도 군위군 군위읍 경북대로 4084 (오곡리),NaN,DC차데모+AC3상+DC콤보,20210101092250,20210101095816,00:29:24,11.95,50.0
2796615,충주휴게소(마산방향),1,충청북도,충주시,충청북도 충주시 중앙탑면 중부내륙고속도로 231 (용전리),NaN,DC차데모+AC3상+DC콤보,20210101095126,20210101095818,00:06:48,4.86,50.0
2796616,내서읍스포츠센터,1,경상남도,창원시,경상남도 창원시 마산회원구 내서읍 중리 391-3,NaN,DC콤보,20210101094752,20210101095819,00:10:27,3.29,100.0
2796617,삼성혈 인근 공영주차장,1,제주특별자치도,제주시,제주특별자치도 제주시 이도1동 1293-1,NaN,DC차데모+AC3상+DC콤보,20210101091323,20210101095819,00:40:00,7.52,50.0
2796618,본오1동행정복지센터,1,경기도,안산시,경기도 안산시 상록구 샘골로 16,NaN,DC콤보,20210101091559,20210101095835,00:40:00,21.66,100.0


In [90]:
df.tail()

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,충전량,충전용량
11665817,죽전휴게소(서울방향),1,경기도,용인시,경기도 용인시 수지구 경부고속도로 400 (죽전동),NaN,DC차데모+AC3상+DC콤보,20220831235918,20220901004034,00:40:00,31.09,급속(50kW)
11665818,원주모범운전자회,21,강원도,원주시,강원도 원주시 단구로 150,NaN,DC콤보,20220831235920,20220901001904,00:19:43,46.51,급속(200kW동시)
11665819,검단2공영주차장,1,인천광역시,서구,인천광역시 서구 당하동 1097-5 검단4동 주민센터 옆,NaN,DC차데모+AC3상+DC콤보,20220831235931,20220901004251,00:40:00,21.97,급속(50kW)
11665820,강당골 공영주차장,1,대구광역시,남구,대구광역시 남구 봉덕동 1272-18,NaN,DC차데모+AC3상+DC콤보,20220831235934,20220901004057,00:40:00,27.60,급속(50kW)
11665821,마천동2공영주차장,1,서울특별시,송파구,서울특별시 송파구 성내천로299-7,NaN,DC차데모+AC3상+DC콤보,20220831235946,20220901003700,00:34:09,9.77,급속(100kW멀티)


In [92]:
# 충전시작일시의 데이터는 년~초 까지 모두 존재
temp = df['충전시작일시'].to_list()
temp2 = [str(i) for i in temp]
count = 0
for one in temp2:
    if len(one) == 8:
        count += 1
print(count)

0


### 충전시작시간 정리

In [93]:
ch_start = df['충전시작일시'].copy()
df['충전시작_년도'] = ch_start.astype(str).str[:4].astype(int)
df['충전시작_월'] = df['충전시작일시'].astype(str).str[4:6].astype(int)
df['충전시작_일'] = df['충전시작일시'].astype(str).str[6:8].astype(int)
df['충전시작_시'] = df['충전시작일시'].astype(str).str[8:10].astype(int)
df['충전시작_분'] = df['충전시작일시'].astype(str).str[10:12].astype(int)
df['충전시작_초'] = df['충전시작일시'].astype(str).str[12:14].astype(int)
df.head()

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,충전량,충전용량,충전시작_년도,충전시작_월,충전시작_일,충전시작_시,충전시작_분,충전시작_초
0,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200117193331,20200117201417,00:40:00,25.47,NaN,2020,1,17,19,33,31
1,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200130143545,20200130150645,00:30:31,17.57,NaN,2020,1,30,14,35,45
2,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200105005728,20200105011329,00:15:35,6.98,NaN,2020,1,5,0,57,28
3,내린천휴게소 양양방향,3,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),100.0,DC콤보,20200120120215,20200120124708,00:41:00,15.30,NaN,2020,1,20,12,2,15
4,한국광기술원,1,광주광역시,북구,광주광역시 북구 첨단 벤처로 108번길 9,50.0,DC차데모+AC3상+DC콤보,20200123085417,20200123092554,00:31:14,21.92,NaN,2020,1,23,8,54,17


In [94]:
df['충전시작일시(변환)'] = df['충전시작_년도'].astype(str) + '-' +  df['충전시작_월'].astype(str) + '-' + df['충전시작_일'].astype(str) + ' ' + df['충전시작_시'].astype(str) + ':' + df['충전시작_분'].astype(str) + ':' + df['충전시작_초'].astype(str)
df.head()

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,충전량,충전용량,충전시작_년도,충전시작_월,충전시작_일,충전시작_시,충전시작_분,충전시작_초,충전시작일시(변환)
0,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200117193331,20200117201417,00:40:00,25.47,NaN,2020,1,17,19,33,31,2020-1-17 19:33:31
1,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200130143545,20200130150645,00:30:31,17.57,NaN,2020,1,30,14,35,45,2020-1-30 14:35:45
2,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200105005728,20200105011329,00:15:35,6.98,NaN,2020,1,5,0,57,28,2020-1-5 0:57:28
3,내린천휴게소 양양방향,3,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),100.0,DC콤보,20200120120215,20200120124708,00:41:00,15.30,NaN,2020,1,20,12,2,15,2020-1-20 12:2:15
4,한국광기술원,1,광주광역시,북구,광주광역시 북구 첨단 벤처로 108번길 9,50.0,DC차데모+AC3상+DC콤보,20200123085417,20200123092554,00:31:14,21.92,NaN,2020,1,23,8,54,17,2020-1-23 8:54:17


In [95]:
df['충전시작일시(변환)'] = pd.to_datetime(df['충전시작일시(변환)'])
df.head()

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,충전량,충전용량,충전시작_년도,충전시작_월,충전시작_일,충전시작_시,충전시작_분,충전시작_초,충전시작일시(변환)
0,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200117193331,20200117201417,00:40:00,25.47,NaN,2020,1,17,19,33,31,2020-01-17 19:33:31
1,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200130143545,20200130150645,00:30:31,17.57,NaN,2020,1,30,14,35,45,2020-01-30 14:35:45
2,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200105005728,20200105011329,00:15:35,6.98,NaN,2020,1,5,0,57,28,2020-01-05 00:57:28
3,내린천휴게소 양양방향,3,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),100.0,DC콤보,20200120120215,20200120124708,00:41:00,15.30,NaN,2020,1,20,12,2,15,2020-01-20 12:02:15
4,한국광기술원,1,광주광역시,북구,광주광역시 북구 첨단 벤처로 108번길 9,50.0,DC차데모+AC3상+DC콤보,20200123085417,20200123092554,00:31:14,21.92,NaN,2020,1,23,8,54,17,2020-01-23 08:54:17


### 충전종료시간 정리

In [96]:
df[df['충전종료일시'] == ' ']

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,충전량,충전용량,충전시작_년도,충전시작_월,충전시작_일,충전시작_시,충전시작_분,충전시작_초,충전시작일시(변환)


In [97]:
# 충전종료일시의 데이터는 년~일까지만 있는 데이터가 71311개 존재
    # 해당 데이터 뒤에 시분초를 '000000'으로 추가하는 작업 필요함.
temp = df['충전종료일시'].to_list()
temp2 = [str(i) for i in temp]
count = 0
for one in temp2:
    if len(one) == 8:
        count += 1
print(count)

71311


In [98]:
df.tail()

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,충전량,충전용량,충전시작_년도,충전시작_월,충전시작_일,충전시작_시,충전시작_분,충전시작_초,충전시작일시(변환)
11665817,죽전휴게소(서울방향),1,경기도,용인시,경기도 용인시 수지구 경부고속도로 400 (죽전동),NaN,DC차데모+AC3상+DC콤보,20220831235918,20220901004034,00:40:00,31.09,급속(50kW),2022,8,31,23,59,18,2022-08-31 23:59:18
11665818,원주모범운전자회,21,강원도,원주시,강원도 원주시 단구로 150,NaN,DC콤보,20220831235920,20220901001904,00:19:43,46.51,급속(200kW동시),2022,8,31,23,59,20,2022-08-31 23:59:20
11665819,검단2공영주차장,1,인천광역시,서구,인천광역시 서구 당하동 1097-5 검단4동 주민센터 옆,NaN,DC차데모+AC3상+DC콤보,20220831235931,20220901004251,00:40:00,21.97,급속(50kW),2022,8,31,23,59,31,2022-08-31 23:59:31
11665820,강당골 공영주차장,1,대구광역시,남구,대구광역시 남구 봉덕동 1272-18,NaN,DC차데모+AC3상+DC콤보,20220831235934,20220901004057,00:40:00,27.60,급속(50kW),2022,8,31,23,59,34,2022-08-31 23:59:34
11665821,마천동2공영주차장,1,서울특별시,송파구,서울특별시 송파구 성내천로299-7,NaN,DC차데모+AC3상+DC콤보,20220831235946,20220901003700,00:34:09,9.77,급속(100kW멀티),2022,8,31,23,59,46,2022-08-31 23:59:46


In [100]:
temp = df['충전종료일시'].to_list()

20220901002054

In [107]:
for i, one in enumerate(temp):
    if len(str(one)) == 8:
        print(i)
        break

1578


In [108]:
temp[1578]

20200118

In [109]:
temp2 = [str(i) for i in temp]
temp2[1578]

'20200118'

In [111]:
for i, one in enumerate(temp2):
    if len(one) == 8:
        temp = one + '000000'
        temp2[i] = temp

In [112]:
temp2[1578]

'20200118000000'

In [113]:
df['충전종료일시'] = temp2
df.head()

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,충전량,충전용량,충전시작_년도,충전시작_월,충전시작_일,충전시작_시,충전시작_분,충전시작_초,충전시작일시(변환)
0,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200117193331,20200117201417,00:40:00,25.47,NaN,2020,1,17,19,33,31,2020-01-17 19:33:31
1,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200130143545,20200130150645,00:30:31,17.57,NaN,2020,1,30,14,35,45,2020-01-30 14:35:45
2,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200105005728,20200105011329,00:15:35,6.98,NaN,2020,1,5,0,57,28,2020-01-05 00:57:28
3,내린천휴게소 양양방향,3,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),100.0,DC콤보,20200120120215,20200120124708,00:41:00,15.30,NaN,2020,1,20,12,2,15,2020-01-20 12:02:15
4,한국광기술원,1,광주광역시,북구,광주광역시 북구 첨단 벤처로 108번길 9,50.0,DC차데모+AC3상+DC콤보,20200123085417,20200123092554,00:31:14,21.92,NaN,2020,1,23,8,54,17,2020-01-23 08:54:17


In [114]:
df.iloc[1578, :]

충전소명                동해웰빙레포츠타운 주차장
충전기ID                          21
지역                            강원도
시군구                           동해시
주소                 강원도 동해시 덕골길 10
충전기용량(kW)                   200.0
충전기타입                        DC콤보
충전시작일시             20200118162253
충전종료일시             20200118000000
충전시간                     00:06:49
충전량                          1.91
충전용량                          NaN
충전시작_년도                      2020
충전시작_월                          1
충전시작_일                         18
충전시작_시                         16
충전시작_분                         22
충전시작_초                         53
충전시작일시(변환)    2020-01-18 16:22:53
Name: 1578, dtype: object

In [126]:
ch_time = df['충전종료일시'].copy()
df['충전종료_년도'] = ch_time.astype(str).str[:4].astype(int)
df['충전종료_월'] = ch_time.astype(str).str[4:6].astype(int)
df['충전종료_일'] = ch_time.astype(str).str[6:8].astype(int)
df['충전종료_시'] = ch_time.astype(str).str[8:10].astype(int)
df['충전종료_분'] = ch_time.astype(str).str[10:12].astype(int)
df['충전종료_초'] = ch_time.astype(str).str[12:14].astype(int)
df.head()

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,...,충전시작_분,충전시작_초,충전시작일시(변환),충전종료_년도,충전종료_월,충전종료_일,충전종료_시,충전종료_분,충전종료_초,충전종료일시(변환)
0,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200117193331,20200117201417,00:40:00,...,33,31,2020-01-17 19:33:31,2020,1,17,20,14,17,2020-1-17 20:14:17
1,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200130143545,20200130150645,00:30:31,...,35,45,2020-01-30 14:35:45,2020,1,30,15,6,45,2020-1-30 15:6:45
2,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200105005728,20200105011329,00:15:35,...,57,28,2020-01-05 00:57:28,2020,1,5,1,13,29,2020-1-5 1:13:29
3,내린천휴게소 양양방향,3,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),100.0,DC콤보,20200120120215,20200120124708,00:41:00,...,2,15,2020-01-20 12:02:15,2020,1,20,12,47,8,2020-1-20 12:47:8
4,한국광기술원,1,광주광역시,북구,광주광역시 북구 첨단 벤처로 108번길 9,50.0,DC차데모+AC3상+DC콤보,20200123085417,20200123092554,00:31:14,...,54,17,2020-01-23 08:54:17,2020,1,23,9,25,54,2020-1-23 9:25:54


In [127]:
df.tail()

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,...,충전시작_분,충전시작_초,충전시작일시(변환),충전종료_년도,충전종료_월,충전종료_일,충전종료_시,충전종료_분,충전종료_초,충전종료일시(변환)
11665817,죽전휴게소(서울방향),1,경기도,용인시,경기도 용인시 수지구 경부고속도로 400 (죽전동),NaN,DC차데모+AC3상+DC콤보,20220831235918,20220901004034,00:40:00,...,59,18,2022-08-31 23:59:18,2022,9,1,0,40,34,2022-9-1 0:40:34
11665818,원주모범운전자회,21,강원도,원주시,강원도 원주시 단구로 150,NaN,DC콤보,20220831235920,20220901001904,00:19:43,...,59,20,2022-08-31 23:59:20,2022,9,1,0,19,4,2022-9-1 0:19:4
11665819,검단2공영주차장,1,인천광역시,서구,인천광역시 서구 당하동 1097-5 검단4동 주민센터 옆,NaN,DC차데모+AC3상+DC콤보,20220831235931,20220901004251,00:40:00,...,59,31,2022-08-31 23:59:31,2022,9,1,0,42,51,2022-9-1 0:42:51
11665820,강당골 공영주차장,1,대구광역시,남구,대구광역시 남구 봉덕동 1272-18,NaN,DC차데모+AC3상+DC콤보,20220831235934,20220901004057,00:40:00,...,59,34,2022-08-31 23:59:34,2022,9,1,0,40,57,2022-9-1 0:40:57
11665821,마천동2공영주차장,1,서울특별시,송파구,서울특별시 송파구 성내천로299-7,NaN,DC차데모+AC3상+DC콤보,20220831235946,20220901003700,00:34:09,...,59,46,2022-08-31 23:59:46,2022,9,1,0,37,0,2022-9-1 0:37:0


In [129]:
df.tail()

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,...,충전시작_분,충전시작_초,충전시작일시(변환),충전종료_년도,충전종료_월,충전종료_일,충전종료_시,충전종료_분,충전종료_초,충전종료일시(변환)
11665817,죽전휴게소(서울방향),1,경기도,용인시,경기도 용인시 수지구 경부고속도로 400 (죽전동),NaN,DC차데모+AC3상+DC콤보,20220831235918,20220901004034,00:40:00,...,59,18,2022-08-31 23:59:18,2022,9,1,0,40,34,2022-9-1 0:40:34
11665818,원주모범운전자회,21,강원도,원주시,강원도 원주시 단구로 150,NaN,DC콤보,20220831235920,20220901001904,00:19:43,...,59,20,2022-08-31 23:59:20,2022,9,1,0,19,4,2022-9-1 0:19:4
11665819,검단2공영주차장,1,인천광역시,서구,인천광역시 서구 당하동 1097-5 검단4동 주민센터 옆,NaN,DC차데모+AC3상+DC콤보,20220831235931,20220901004251,00:40:00,...,59,31,2022-08-31 23:59:31,2022,9,1,0,42,51,2022-9-1 0:42:51
11665820,강당골 공영주차장,1,대구광역시,남구,대구광역시 남구 봉덕동 1272-18,NaN,DC차데모+AC3상+DC콤보,20220831235934,20220901004057,00:40:00,...,59,34,2022-08-31 23:59:34,2022,9,1,0,40,57,2022-9-1 0:40:57
11665821,마천동2공영주차장,1,서울특별시,송파구,서울특별시 송파구 성내천로299-7,NaN,DC차데모+AC3상+DC콤보,20220831235946,20220901003700,00:34:09,...,59,46,2022-08-31 23:59:46,2022,9,1,0,37,0,2022-9-1 0:37:0


#### 충전종료일시-분 오류 데이터
- 3244개

In [131]:
df[df['충전종료_분'] >= 60]

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,...,충전시작_분,충전시작_초,충전시작일시(변환),충전종료_년도,충전종료_월,충전종료_일,충전종료_시,충전종료_분,충전종료_초,충전종료일시(변환)
1964993,포천시의회 야외주차장,45,경기도,포천시,경기도 포천시 중앙로 87,100.0,DC콤보,20201023174301,20201023176497,00:36:36,...,43,1,2020-10-23 17:43:01,2020,10,23,17,64,97,2020-10-23 17:64:97
1966556,포천시의회 야외주차장,44,경기도,포천시,경기도 포천시 중앙로 87,100.0,DC콤보,20201023105759,20201023106864,00:18:25,...,57,59,2020-10-23 10:57:59,2020,10,23,10,68,64,2020-10-23 10:68:64
1968601,포천시의회 야외주차장,43,경기도,포천시,경기도 포천시 중앙로 87,100.0,DC콤보,20201023113935,20201023116335,00:40:00,...,39,35,2020-10-23 11:39:35,2020,10,23,11,63,35,2020-10-23 11:63:35
1968849,포천시의회 야외주차장,41,경기도,포천시,경기도 포천시 중앙로 87,100.0,DC콤보,20201023155644,20201023158044,00:40:00,...,56,44,2020-10-23 15:56:44,2020,10,23,15,80,44,2020-10-23 15:80:44
1970888,포천시의회 야외주차장,41,경기도,포천시,경기도 포천시 중앙로 87,100.0,DC콤보,20201023165300,20201023167700,00:40:00,...,53,0,2020-10-23 16:53:00,2020,10,23,16,77,0,2020-10-23 16:77:0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4243822,웅전공영주차장,1,제주특별자치도,제주시,제주특별자치도 제주시 노형동 2517-1,NaN,DC콤보,20210503173914,20210503176314,00:40:00,...,39,14,2021-05-03 17:39:14,2021,5,3,17,63,14,2021-5-3 17:63:14
4244731,수덕공영주차장,1,제주특별자치도,제주시,제주특별자치도 제주시 노형동 2534-3,NaN,DC콤보,20210502144657,20210502146537,00:31:20,...,46,57,2021-05-02 14:46:57,2021,5,2,14,65,37,2021-5-2 14:65:37
4244805,웅전공영주차장,1,제주특별자치도,제주시,제주특별자치도 제주시 노형동 2517-1,NaN,DC콤보,20210502164613,20210502166324,00:28:31,...,46,13,2021-05-02 16:46:13,2021,5,2,16,63,24,2021-5-2 16:63:24
4245720,수덕공영주차장,1,제주특별자치도,제주시,제주특별자치도 제주시 노형동 2534-3,NaN,DC콤보,20210501124030,20210501126430,00:40:00,...,40,30,2021-05-01 12:40:30,2021,5,1,12,64,30,2021-5-1 12:64:30


#### 충전종료일시-초 오류 데이터
- 1495개

In [132]:
df[df['충전종료_초'] >= 60]

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,...,충전시작_분,충전시작_초,충전시작일시(변환),충전종료_년도,충전종료_월,충전종료_일,충전종료_시,충전종료_분,충전종료_초,충전종료일시(변환)
1960560,포천시의회 야외주차장,44,경기도,포천시,경기도 포천시 중앙로 87,100.0,DC콤보,20201023102909,20201023103099,00:03:10,...,29,9,2020-10-23 10:29:09,2020,10,23,10,30,99,2020-10-23 10:30:99
1964993,포천시의회 야외주차장,45,경기도,포천시,경기도 포천시 중앙로 87,100.0,DC콤보,20201023174301,20201023176497,00:36:36,...,43,1,2020-10-23 17:43:01,2020,10,23,17,64,97,2020-10-23 17:64:97
1966556,포천시의회 야외주차장,44,경기도,포천시,경기도 포천시 중앙로 87,100.0,DC콤보,20201023105759,20201023106864,00:18:25,...,57,59,2020-10-23 10:57:59,2020,10,23,10,68,64,2020-10-23 10:68:64
1976775,포천시의회 야외주차장,42,경기도,포천시,경기도 포천시 중앙로 87,100.0,DC콤보,20201023161021,20201023163292,00:37:51,...,10,21,2020-10-23 16:10:21,2020,10,23,16,32,92,2020-10-23 16:32:92
2043371,포천시의회 야외주차장,43,경기도,포천시,경기도 포천시 중앙로 87,100.0,DC콤보,20201027110722,20201027112396,00:27:54,...,7,22,2020-10-27 11:07:22,2020,10,27,11,23,96,2020-10-27 11:23:96
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4240162,수덕공영주차장,1,제주특별자치도,제주시,제주특별자치도 제주시 노형동 2534-3,NaN,DC콤보,20210503014439,20210503015192,00:12:33,...,44,39,2021-05-03 01:44:39,2021,5,3,1,51,92,2021-5-3 1:51:92
4240310,용두암 공영주차장,1,제주특별자치도,제주시,제주특별자치도 제주시 용두암길15,NaN,DC콤보,20210503154020,20210503155676,00:27:36,...,40,20,2021-05-03 15:40:20,2021,5,3,15,56,76,2021-5-3 15:56:76
4240738,웅전공영주차장,1,제주특별자치도,제주시,제주특별자치도 제주시 노형동 2517-1,NaN,DC콤보,20210501064956,20210501066797,00:30:41,...,49,56,2021-05-01 06:49:56,2021,5,1,6,67,97,2021-5-1 6:67:97
4242576,수덕공영주차장,1,제주특별자치도,제주시,제주특별자치도 제주시 노형동 2534-3,NaN,DC콤보,20210501145800,20210501146795,00:16:35,...,58,0,2021-05-01 14:58:00,2021,5,1,14,67,95,2021-5-1 14:67:95


##### 오류 데이터 수정
- 60분 -> +1시간
- 60초 -> +1분

In [135]:
over_m_idx = df[df['충전종료_분'] >= 60].index
over_m_idx

Int64Index([1964993, 1966556, 1968601, 1968849, 1970888, 1972692, 1977670,
            1979247, 1985718, 1989454,
            ...
            4238216, 4240734, 4240738, 4242537, 4242576, 4243822, 4244731,
            4244805, 4245720, 4245930],
           dtype='int64', length=3244)

In [136]:
for i in over_m_idx:
    df.loc[i, '충전종료_시'] += 1
    df.loc[i, '충전종료_분'] -= 60

In [137]:
df.iloc[1964993, :]

충전소명                  포천시의회 야외주차장
충전기ID                          45
지역                            경기도
시군구                           포천시
주소                 경기도 포천시 중앙로 87
충전기용량(kW)                   100.0
충전기타입                        DC콤보
충전시작일시             20201023174301
충전종료일시             20201023176497
충전시간                     00:36:36
충전량                         25.44
충전용량                          NaN
충전시작_년도                      2020
충전시작_월                         10
충전시작_일                         23
충전시작_시                         17
충전시작_분                         43
충전시작_초                          1
충전시작일시(변환)    2020-10-23 17:43:01
충전종료_년도                      2020
충전종료_월                         10
충전종료_일                         23
충전종료_시                         18
충전종료_분                          4
충전종료_초                         97
충전종료일시(변환)    2020-10-23 17:64:97
Name: 1964993, dtype: object

In [138]:
df.tail()

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,...,충전시작_분,충전시작_초,충전시작일시(변환),충전종료_년도,충전종료_월,충전종료_일,충전종료_시,충전종료_분,충전종료_초,충전종료일시(변환)
11665817,죽전휴게소(서울방향),1,경기도,용인시,경기도 용인시 수지구 경부고속도로 400 (죽전동),NaN,DC차데모+AC3상+DC콤보,20220831235918,20220901004034,00:40:00,...,59,18,2022-08-31 23:59:18,2022,9,1,0,40,34,2022-9-1 0:40:34
11665818,원주모범운전자회,21,강원도,원주시,강원도 원주시 단구로 150,NaN,DC콤보,20220831235920,20220901001904,00:19:43,...,59,20,2022-08-31 23:59:20,2022,9,1,0,19,4,2022-9-1 0:19:4
11665819,검단2공영주차장,1,인천광역시,서구,인천광역시 서구 당하동 1097-5 검단4동 주민센터 옆,NaN,DC차데모+AC3상+DC콤보,20220831235931,20220901004251,00:40:00,...,59,31,2022-08-31 23:59:31,2022,9,1,0,42,51,2022-9-1 0:42:51
11665820,강당골 공영주차장,1,대구광역시,남구,대구광역시 남구 봉덕동 1272-18,NaN,DC차데모+AC3상+DC콤보,20220831235934,20220901004057,00:40:00,...,59,34,2022-08-31 23:59:34,2022,9,1,0,40,57,2022-9-1 0:40:57
11665821,마천동2공영주차장,1,서울특별시,송파구,서울특별시 송파구 성내천로299-7,NaN,DC차데모+AC3상+DC콤보,20220831235946,20220901003700,00:34:09,...,59,46,2022-08-31 23:59:46,2022,9,1,0,37,0,2022-9-1 0:37:0


In [139]:
over_s_idx = df[df['충전종료_초'] >= 60].index
over_s_idx

Int64Index([1960560, 1964993, 1966556, 1976775, 2043371, 2044340, 2049489,
            2053437, 2067815, 2073197,
            ...
            4237800, 4238216, 4238983, 4239703, 4240023, 4240162, 4240310,
            4240738, 4242576, 4245891],
           dtype='int64', length=1495)

In [140]:
for i in over_s_idx:
    df.loc[i, '충전종료_분'] += 1
    df.loc[i, '충전종료_초'] -= 60

In [142]:
df.iloc[1960560, :]

충전소명                  포천시의회 야외주차장
충전기ID                          44
지역                            경기도
시군구                           포천시
주소                 경기도 포천시 중앙로 87
충전기용량(kW)                   100.0
충전기타입                        DC콤보
충전시작일시             20201023102909
충전종료일시             20201023103099
충전시간                     00:03:10
충전량                          2.36
충전용량                          NaN
충전시작_년도                      2020
충전시작_월                         10
충전시작_일                         23
충전시작_시                         10
충전시작_분                         29
충전시작_초                          9
충전시작일시(변환)    2020-10-23 10:29:09
충전종료_년도                      2020
충전종료_월                         10
충전종료_일                         23
충전종료_시                         10
충전종료_분                         31
충전종료_초                         39
충전종료일시(변환)    2020-10-23 10:30:99
Name: 1960560, dtype: object

In [145]:
df[df['충전종료_시'] >= 24]

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,...,충전시작_분,충전시작_초,충전시작일시(변환),충전종료_년도,충전종료_월,충전종료_일,충전종료_시,충전종료_분,충전종료_초,충전종료일시(변환)
2168170,제천종합운동장 대형버스 주차장,41,충청북도,제천시,충청북도 제천시 숭의로 101,100.0,DC콤보,20201030234823,20201030237223,00:40:00,...,48,23,2020-10-30 23:48:23,2020,10,30,24,12,23,2020-10-30 24:12:23
2171801,서산동부전통시장 공영주차장,43,충청남도,서산시,충청남도 서산시 동문동 800번지 -,100.0,DC콤보,20201103235556,20201103237956,00:40:00,...,55,56,2020-11-03 23:55:56,2020,11,3,24,19,56,2020-11-3 24:19:56
2189569,대구스타디움 주차장,42,대구광역시,수성구,대구광역시 수성구 유니버시아드로 180,100.0,DC콤보,20201103234942,20201103237181,00:37:19,...,49,42,2020-11-03 23:49:42,2020,11,3,24,12,21,2020-11-3 24:12:21
2189571,롯데마트 중계점 야외주차장,45,서울특별시,노원구,서울특별시 노원구 노원로 330,100.0,DC콤보,20201103234432,20201103236832,00:40:00,...,44,32,2020-11-03 23:44:32,2020,11,3,24,8,32,2020-11-3 24:8:32
2192634,대구봉무공원 주차장,44,대구광역시,동구,대구광역시 동구 봉무동 산 138-5,100.0,DC콤보,20201104234049,20201104236449,00:40:00,...,40,49,2020-11-04 23:40:49,2020,11,4,24,4,49,2020-11-4 24:4:49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3783366,용두암 공영주차장,1,제주특별자치도,제주시,제주특별자치도 제주시 용두암길15,NaN,DC콤보,20210411233845,20210411236245,00:40:00,...,38,45,2021-04-11 23:38:45,2021,4,11,24,2,45,2021-4-11 24:2:45
3787919,수덕공영주차장,1,제주특별자치도,제주시,제주특별자치도 제주시 노형동 2534-3,NaN,DC콤보,20210419235407,20210419236378,00:16:11,...,54,7,2021-04-19 23:54:07,2021,4,19,24,4,18,2021-4-19 24:4:18
3900443,웅전공영주차장,1,제주특별자치도,제주시,제주특별자치도 제주시 노형동 2517-1,NaN,DC콤보,20210417235151,20210417236194,00:17:23,...,51,51,2021-04-17 23:51:51,2021,4,17,24,2,34,2021-4-17 24:2:34
3967294,웅전공영주차장,1,제주특별자치도,제주시,제주특별자치도 제주시 노형동 2517-1,NaN,DC콤보,20210504234728,20210504237128,00:40:00,...,47,28,2021-05-04 23:47:28,2021,5,4,24,11,28,2021-5-4 24:11:28


#### 충전종료일시-시 오류 데이터
- 70개

In [146]:
over_h_idx = df[df['충전종료_시'] >= 24].index
over_h_idx

Int64Index([2168170, 2171801, 2189569, 2189571, 2192634, 2208104, 2208113,
            2210088, 2210097, 2225369, 2225386, 2279227, 2286662, 2292717,
            2318768, 2319950, 2323394, 2352145, 2369004, 2440150, 2584466,
            2629617, 2643620, 2646234, 2667512, 2673846, 2711022, 2774369,
            2775050, 2805642, 2826716, 2869482, 2875336, 2876720, 2881192,
            2927893, 2929332, 2937690, 2937716, 2957597, 2987779, 3007178,
            3019091, 3033859, 3073494, 3078599, 3154875, 3165642, 3168914,
            3178385, 3184928, 3212925, 3232880, 3300780, 3307508, 3309499,
            3443211, 3443229, 3474185, 3539531, 3539611, 3656645, 3733232,
            3733790, 3782239, 3783366, 3787919, 3900443, 3967294, 4226894],
           dtype='int64')

In [147]:
for i in over_h_idx:
    df.loc[i, '충전종료_일'] += 1
    df.loc[i, '충전종료_시'] -= 24

In [148]:
df.iloc[2168170, :]

충전소명             제천종합운동장 대형버스 주차장
충전기ID                          41
지역                           충청북도
시군구                           제천시
주소               충청북도 제천시 숭의로 101
충전기용량(kW)                   100.0
충전기타입                        DC콤보
충전시작일시             20201030234823
충전종료일시             20201030237223
충전시간                     00:40:00
충전량                         23.22
충전용량                          NaN
충전시작_년도                      2020
충전시작_월                         10
충전시작_일                         30
충전시작_시                         23
충전시작_분                         48
충전시작_초                         23
충전시작일시(변환)    2020-10-30 23:48:23
충전종료_년도                      2020
충전종료_월                         10
충전종료_일                         31
충전종료_시                          0
충전종료_분                         12
충전종료_초                         23
충전종료일시(변환)    2020-10-30 24:12:23
Name: 2168170, dtype: object

#### 충전종료일시-일 오류 데이터
- 2개

In [154]:
df[(df['충전종료_월'] == 1) & (df['충전종료_일'] > 31)].index

Int64Index([], dtype='int64')

In [165]:
# 2020년은 윤년
df[(df['충전종료_월'] == 2) & (df['충전종료_일'] > 29)].index

Int64Index([], dtype='int64')

In [155]:
df[(df['충전종료_월'] == 3) & (df['충전종료_일'] > 31)].index

Int64Index([], dtype='int64')

In [156]:
df[(df['충전종료_월'] == 4) & (df['충전종료_일'] > 30)].index

Int64Index([], dtype='int64')

In [157]:
df[(df['충전종료_월'] == 5) & (df['충전종료_일'] > 31)].index

Int64Index([], dtype='int64')

In [158]:
df[(df['충전종료_월'] == 6) & (df['충전종료_일'] > 31)].index

Int64Index([], dtype='int64')

In [159]:
df[(df['충전종료_월'] == 7) & (df['충전종료_일'] > 31)].index

Int64Index([], dtype='int64')

In [160]:
df[(df['충전종료_월'] == 8) & (df['충전종료_일'] > 31)].index

Int64Index([], dtype='int64')

In [161]:
df[(df['충전종료_월'] == 9) & (df['충전종료_일'] > 30)].index

Int64Index([], dtype='int64')

In [162]:
df[(df['충전종료_월'] == 10) & (df['충전종료_일'] > 31)].index

Int64Index([], dtype='int64')

In [163]:
df[(df['충전종료_월'] == 11) & (df['충전종료_일'] > 30)].index

Int64Index([2440150], dtype='int64')

In [164]:
df[(df['충전종료_월'] == 12) & (df['충전종료_일'] > 31)].index

Int64Index([2775050], dtype='int64')

In [166]:
df.iloc[2440150, :]

충전소명                         웅전공영주차장
충전기ID                              1
지역                           제주특별자치도
시군구                              제주시
주소            제주특별자치도 제주시 노형동 2517-1
충전기용량(kW)                      100.0
충전기타입                           DC콤보
충전시작일시                20201130235748
충전종료일시                20201130238148
충전시간                        00:40:00
충전량                             8.44
충전용량                             NaN
충전시작_년도                         2020
충전시작_월                            11
충전시작_일                            30
충전시작_시                            23
충전시작_분                            57
충전시작_초                            48
충전시작일시(변환)       2020-11-30 23:57:48
충전종료_년도                         2020
충전종료_월                            11
충전종료_일                            31
충전종료_시                             0
충전종료_분                            21
충전종료_초                            48
충전종료일시(변환)       2020-11-30 24:21:48
Name: 2440150, dtype: object

In [167]:
df.loc[2440150, '충전종료_월'] = 12
df.loc[2440150, '충전종료_일'] = 1
df.iloc[2440150, :]

충전소명                         웅전공영주차장
충전기ID                              1
지역                           제주특별자치도
시군구                              제주시
주소            제주특별자치도 제주시 노형동 2517-1
충전기용량(kW)                      100.0
충전기타입                           DC콤보
충전시작일시                20201130235748
충전종료일시                20201130238148
충전시간                        00:40:00
충전량                             8.44
충전용량                             NaN
충전시작_년도                         2020
충전시작_월                            11
충전시작_일                            30
충전시작_시                            23
충전시작_분                            57
충전시작_초                            48
충전시작일시(변환)       2020-11-30 23:57:48
충전종료_년도                         2020
충전종료_월                            12
충전종료_일                             1
충전종료_시                             0
충전종료_분                            21
충전종료_초                            48
충전종료일시(변환)       2020-11-30 24:21:48
Name: 2440150, dtype: object

In [168]:
df.iloc[2775050, :]

충전소명               롯데마트 중계점 야외주차장
충전기ID                          41
지역                          서울특별시
시군구                           노원구
주소             서울특별시 노원구 노원로  330
충전기용량(kW)                   100.0
충전기타입                        DC콤보
충전시작일시             20201231234219
충전종료일시             20201231236619
충전시간                     00:40:00
충전량                         28.09
충전용량                          NaN
충전시작_년도                      2020
충전시작_월                         12
충전시작_일                         31
충전시작_시                         23
충전시작_분                         42
충전시작_초                         19
충전시작일시(변환)    2020-12-31 23:42:19
충전종료_년도                      2020
충전종료_월                         12
충전종료_일                         32
충전종료_시                          0
충전종료_분                          6
충전종료_초                         19
충전종료일시(변환)     2020-12-31 24:6:19
Name: 2775050, dtype: object

In [169]:
df.loc[2775050, '충전종료_년도'] = 2021
df.loc[2775050, '충전종료_월'] = 1
df.loc[2775050, '충전종료_일'] = 1
df.iloc[2775050, :]

충전소명               롯데마트 중계점 야외주차장
충전기ID                          41
지역                          서울특별시
시군구                           노원구
주소             서울특별시 노원구 노원로  330
충전기용량(kW)                   100.0
충전기타입                        DC콤보
충전시작일시             20201231234219
충전종료일시             20201231236619
충전시간                     00:40:00
충전량                         28.09
충전용량                          NaN
충전시작_년도                      2020
충전시작_월                         12
충전시작_일                         31
충전시작_시                         23
충전시작_분                         42
충전시작_초                         19
충전시작일시(변환)    2020-12-31 23:42:19
충전종료_년도                      2021
충전종료_월                          1
충전종료_일                          1
충전종료_시                          0
충전종료_분                          6
충전종료_초                         19
충전종료일시(변환)     2020-12-31 24:6:19
Name: 2775050, dtype: object

### 충전종료일시-분 오류 데이터
- 26개

In [173]:
df[df['충전종료_분'] >= 60].shape

(26, 26)

In [174]:
over_m_idx = df[df['충전종료_분'] >= 60].index
over_m_idx

Int64Index([2174488, 2183068, 2184405, 2186995, 2187511, 2189605, 2192044,
            2199843, 2240070, 2240074, 2290964, 2315988, 2328704, 2359633,
            2370228, 2756009, 2826101, 2942980, 3020539, 3022887, 3466767,
            3554013, 3781802, 3799421, 3957630, 4233811],
           dtype='int64')

In [175]:
for i in over_m_idx:
    df.loc[i, '충전종료_시'] += 1
    df.loc[i, '충전종료_분'] -= 60

#### 충전종료일시-시 오류 데이터
- 1개

In [176]:
over_h_idx = df[df['충전종료_시'] >= 24].index
over_h_idx

Int64Index([3022887], dtype='int64')

In [177]:
df.iloc[3022887, :]

충전소명               롯데마트 중계점 야외주차장
충전기ID                          41
지역                          서울특별시
시군구                           노원구
주소             서울특별시 노원구 노원로  330
충전기용량(kW)                     NaN
충전기타입                        DC콤보
충전시작일시             20210113233756
충전종료일시             20210113235982
충전시간                     00:37:06
충전량                         13.69
충전용량                        100.0
충전시작_년도                      2021
충전시작_월                          1
충전시작_일                         13
충전시작_시                         23
충전시작_분                         37
충전시작_초                         56
충전시작일시(변환)    2021-01-13 23:37:56
충전종료_년도                      2021
충전종료_월                          1
충전종료_일                         13
충전종료_시                         24
충전종료_분                          0
충전종료_초                         22
충전종료일시(변환)     2021-1-13 23:60:22
Name: 3022887, dtype: object

In [178]:
for i in over_h_idx:
    df.loc[i, '충전종료_일'] += 1
    df.loc[i, '충전종료_시'] -= 24

#### 충전종료일시-일 오류 데이터
- 2개

In [181]:
df[(df['충전종료_월'] == 1) & (df['충전종료_일'] > 31)].index

Int64Index([], dtype='int64')

In [183]:
# 2020년은 윤년
df[(df['충전종료_년도'] == 2020) & (df['충전종료_월'] == 2) & (df['충전종료_일'] > 29)].index

Int64Index([], dtype='int64')

In [184]:
df[(df['충전종료_년도'] == 2021) & (df['충전종료_월'] == 2) & (df['충전종료_일'] > 28)].index

Int64Index([3232880], dtype='int64')

In [185]:
df[(df['충전종료_년도'] == 2022) & (df['충전종료_월'] == 2) & (df['충전종료_일'] > 28)].index

Int64Index([], dtype='int64')

In [186]:
df[(df['충전종료_월'] == 3) & (df['충전종료_일'] > 31)].index

Int64Index([], dtype='int64')

In [187]:
df[(df['충전종료_월'] == 4) & (df['충전종료_일'] > 30)].index

Int64Index([], dtype='int64')

In [188]:
df[(df['충전종료_월'] == 5) & (df['충전종료_일'] > 31)].index

Int64Index([], dtype='int64')

In [189]:
df[(df['충전종료_월'] == 6) & (df['충전종료_일'] > 31)].index

Int64Index([], dtype='int64')

In [190]:
df[(df['충전종료_월'] == 7) & (df['충전종료_일'] > 31)].index

Int64Index([], dtype='int64')

In [191]:
df[(df['충전종료_월'] == 8) & (df['충전종료_일'] > 31)].index

Int64Index([], dtype='int64')

In [192]:
df[(df['충전종료_월'] == 9) & (df['충전종료_일'] > 30)].index

Int64Index([], dtype='int64')

In [193]:
df[(df['충전종료_월'] == 10) & (df['충전종료_일'] > 31)].index

Int64Index([], dtype='int64')

In [194]:
df[(df['충전종료_월'] == 11) & (df['충전종료_일'] > 30)].index

Int64Index([], dtype='int64')

In [195]:
df[(df['충전종료_월'] == 12) & (df['충전종료_일'] > 31)].index

Int64Index([], dtype='int64')

In [196]:
df.iloc[3232880, :]

충전소명                         웅전공영주차장
충전기ID                              1
지역                           제주특별자치도
시군구                              제주시
주소            제주특별자치도 제주시 노형동 2517-1
충전기용량(kW)                        NaN
충전기타입                           DC콤보
충전시작일시                20210228234752
충전종료일시                20210228237152
충전시간                        00:40:00
충전량                            23.72
충전용량                           100.0
충전시작_년도                         2021
충전시작_월                             2
충전시작_일                            28
충전시작_시                            23
충전시작_분                            47
충전시작_초                            52
충전시작일시(변환)       2021-02-28 23:47:52
충전종료_년도                         2021
충전종료_월                             2
충전종료_일                            29
충전종료_시                             0
충전종료_분                            11
충전종료_초                            52
충전종료일시(변환)         2021-2-29 0:11:52
Name: 3232880, dtype: object

In [197]:
df.loc[3232880, '충전종료_월'] = 3
df.loc[3232880, '충전종료_일'] = 1
df.iloc[3232880, :]

충전소명                         웅전공영주차장
충전기ID                              1
지역                           제주특별자치도
시군구                              제주시
주소            제주특별자치도 제주시 노형동 2517-1
충전기용량(kW)                        NaN
충전기타입                           DC콤보
충전시작일시                20210228234752
충전종료일시                20210228237152
충전시간                        00:40:00
충전량                            23.72
충전용량                           100.0
충전시작_년도                         2021
충전시작_월                             2
충전시작_일                            28
충전시작_시                            23
충전시작_분                            47
충전시작_초                            52
충전시작일시(변환)       2021-02-28 23:47:52
충전종료_년도                         2021
충전종료_월                             3
충전종료_일                             1
충전종료_시                             0
충전종료_분                            11
충전종료_초                            52
충전종료일시(변환)         2021-2-29 0:11:52
Name: 3232880, dtype: object

#### 충전종료시간(변환)

In [198]:
df['충전종료일시(변환)'] = df['충전종료_년도'].astype(str) + '-' +  df['충전종료_월'].astype(str) + '-' + df['충전종료_일'].astype(str) + ' ' + df['충전종료_시'].astype(str) + ':' + df['충전종료_분'].astype(str) + ':' + df['충전종료_초'].astype(str)
df.head()

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,...,충전시작_분,충전시작_초,충전시작일시(변환),충전종료_년도,충전종료_월,충전종료_일,충전종료_시,충전종료_분,충전종료_초,충전종료일시(변환)
0,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200117193331,20200117201417,00:40:00,...,33,31,2020-01-17 19:33:31,2020,1,17,20,14,17,2020-1-17 20:14:17
1,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200130143545,20200130150645,00:30:31,...,35,45,2020-01-30 14:35:45,2020,1,30,15,6,45,2020-1-30 15:6:45
2,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200105005728,20200105011329,00:15:35,...,57,28,2020-01-05 00:57:28,2020,1,5,1,13,29,2020-1-5 1:13:29
3,내린천휴게소 양양방향,3,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),100.0,DC콤보,20200120120215,20200120124708,00:41:00,...,2,15,2020-01-20 12:02:15,2020,1,20,12,47,8,2020-1-20 12:47:8
4,한국광기술원,1,광주광역시,북구,광주광역시 북구 첨단 벤처로 108번길 9,50.0,DC차데모+AC3상+DC콤보,20200123085417,20200123092554,00:31:14,...,54,17,2020-01-23 08:54:17,2020,1,23,9,25,54,2020-1-23 9:25:54


In [199]:
df['충전종료일시(변환)'] = pd.to_datetime(df['충전종료일시(변환)'])
df.head()

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,...,충전시작_분,충전시작_초,충전시작일시(변환),충전종료_년도,충전종료_월,충전종료_일,충전종료_시,충전종료_분,충전종료_초,충전종료일시(변환)
0,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200117193331,20200117201417,00:40:00,...,33,31,2020-01-17 19:33:31,2020,1,17,20,14,17,2020-01-17 20:14:17
1,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200130143545,20200130150645,00:30:31,...,35,45,2020-01-30 14:35:45,2020,1,30,15,6,45,2020-01-30 15:06:45
2,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200105005728,20200105011329,00:15:35,...,57,28,2020-01-05 00:57:28,2020,1,5,1,13,29,2020-01-05 01:13:29
3,내린천휴게소 양양방향,3,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),100.0,DC콤보,20200120120215,20200120124708,00:41:00,...,2,15,2020-01-20 12:02:15,2020,1,20,12,47,8,2020-01-20 12:47:08
4,한국광기술원,1,광주광역시,북구,광주광역시 북구 첨단 벤처로 108번길 9,50.0,DC차데모+AC3상+DC콤보,20200123085417,20200123092554,00:31:14,...,54,17,2020-01-23 08:54:17,2020,1,23,9,25,54,2020-01-23 09:25:54


In [200]:
df['충전시간(계산)'] = df['충전종료일시(변환)'] - df['충전시작일시(변환)']
df.head()

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,...,충전시작_초,충전시작일시(변환),충전종료_년도,충전종료_월,충전종료_일,충전종료_시,충전종료_분,충전종료_초,충전종료일시(변환),충전시간(계산)
0,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200117193331,20200117201417,00:40:00,...,31,2020-01-17 19:33:31,2020,1,17,20,14,17,2020-01-17 20:14:17,0 days 00:40:46
1,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200130143545,20200130150645,00:30:31,...,45,2020-01-30 14:35:45,2020,1,30,15,6,45,2020-01-30 15:06:45,0 days 00:31:00
2,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200105005728,20200105011329,00:15:35,...,28,2020-01-05 00:57:28,2020,1,5,1,13,29,2020-01-05 01:13:29,0 days 00:16:01
3,내린천휴게소 양양방향,3,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),100.0,DC콤보,20200120120215,20200120124708,00:41:00,...,15,2020-01-20 12:02:15,2020,1,20,12,47,8,2020-01-20 12:47:08,0 days 00:44:53
4,한국광기술원,1,광주광역시,북구,광주광역시 북구 첨단 벤처로 108번길 9,50.0,DC차데모+AC3상+DC콤보,20200123085417,20200123092554,00:31:14,...,17,2020-01-23 08:54:17,2020,1,23,9,25,54,2020-01-23 09:25:54,0 days 00:31:37


In [204]:
df[df['충전시작_년도'] == 2020].tail()

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,...,충전시작_초,충전시작일시(변환),충전종료_년도,충전종료_월,충전종료_일,충전종료_시,충전종료_분,충전종료_초,충전종료일시(변환),충전시간(계산)
2796609,탄천제2호 공영주차장,1,서울특별시,강남구,"서울특별시 강남구 일원동 4-49 , 주차장 입구 오른쪽",50.0,DC차데모+AC3상+DC콤보,20201219120138,20201219124144,00:40:00,...,38,2020-12-19 12:01:38,2020,12,19,12,41,44,2020-12-19 12:41:44,0 days 00:40:06
2796610,온천장역 공영주차장,1,부산광역시,동래구,부산광역시 동래구 중앙대로 1495-10 공영주차장,50.0,DC차데모+AC3상+DC콤보,20201219120210,20201219124217,00:40:00,...,10,2020-12-19 12:02:10,2020,12,19,12,42,17,2020-12-19 12:42:17,0 days 00:40:07
2796611,화서휴게소(상주방향),1,경상북도,상주시,경상북도 상주시 화서면 터골길 51-25,50.0,DC차데모+AC3상+DC콤보,20201219120513,20201219124254,00:37:28,...,13,2020-12-19 12:05:13,2020,12,19,12,42,54,2020-12-19 12:42:54,0 days 00:37:41
2796612,삼양동주민센터,1,제주특별자치도,제주시,"제주특별자치도 제주시 지석13길 6 , 주차장",50.0,DC차데모+AC3상+DC콤보,20201219121419,20201219124313,00:28:47,...,19,2020-12-19 12:14:19,2020,12,19,12,43,13,2020-12-19 12:43:13,0 days 00:28:54
2796613,거창휴게소(옥포방향),1,경상남도,거창군,경상남도 거창군 가조면 도리,50.0,DC차데모+AC3상+DC콤보,20201219120725,20201219124405,00:36:34,...,25,2020-12-19 12:07:25,2020,12,19,12,44,5,2020-12-19 12:44:05,0 days 00:36:40


In [205]:
df[df['충전시작_년도'] == 2021].head()

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,...,충전시작_초,충전시작일시(변환),충전종료_년도,충전종료_월,충전종료_일,충전종료_시,충전종료_분,충전종료_초,충전종료일시(변환),충전시간(계산)
2796614,군위휴게소(부산방향),1,경상북도,군위군,경상북도 군위군 군위읍 경북대로 4084 (오곡리),NaN,DC차데모+AC3상+DC콤보,20210101092250,20210101095816,00:29:24,...,50,2021-01-01 09:22:50,2021,1,1,9,58,16,2021-01-01 09:58:16,0 days 00:35:26
2796615,충주휴게소(마산방향),1,충청북도,충주시,충청북도 충주시 중앙탑면 중부내륙고속도로 231 (용전리),NaN,DC차데모+AC3상+DC콤보,20210101095126,20210101095818,00:06:48,...,26,2021-01-01 09:51:26,2021,1,1,9,58,18,2021-01-01 09:58:18,0 days 00:06:52
2796616,내서읍스포츠센터,1,경상남도,창원시,경상남도 창원시 마산회원구 내서읍 중리 391-3,NaN,DC콤보,20210101094752,20210101095819,00:10:27,...,52,2021-01-01 09:47:52,2021,1,1,9,58,19,2021-01-01 09:58:19,0 days 00:10:27
2796617,삼성혈 인근 공영주차장,1,제주특별자치도,제주시,제주특별자치도 제주시 이도1동 1293-1,NaN,DC차데모+AC3상+DC콤보,20210101091323,20210101095819,00:40:00,...,23,2021-01-01 09:13:23,2021,1,1,9,58,19,2021-01-01 09:58:19,0 days 00:44:56
2796618,본오1동행정복지센터,1,경기도,안산시,경기도 안산시 상록구 샘골로 16,NaN,DC콤보,20210101091559,20210101095835,00:40:00,...,59,2021-01-01 09:15:59,2021,1,1,9,58,35,2021-01-01 09:58:35,0 days 00:42:36


### 충전기용량과 충전용량 합치기

In [206]:
df2 = df.copy()

In [209]:
temp = df2[df2['충전시작_년도'] == 2020]['충전기용량(kW)'].to_list()
temp[-5:]

[50.0, 50.0, 50.0, 50.0, 50.0]

In [211]:
len(temp)

2796614

In [210]:
temp2 = df2[df2['충전시작_년도'] == 2021]['충전용량'].to_list()
temp2[-5:]

[50.0, 50.0, 50.0, 50.0, 50.0]

In [212]:
len(temp2)

4259845

In [213]:
temp3 = df2[df2['충전시작_년도'] == 2022]['충전용량'].to_list()
temp3[-5:]

['급속(50kW)', '급속(200kW동시)', '급속(50kW)', '급속(50kW)', '급속(100kW멀티)']

In [214]:
len(temp3)

4609363

In [215]:
len(temp + temp2 + temp3)

11665822

In [216]:
df['충전용량'] = temp + temp2 + temp3
df.head()

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,...,충전시작_초,충전시작일시(변환),충전종료_년도,충전종료_월,충전종료_일,충전종료_시,충전종료_분,충전종료_초,충전종료일시(변환),충전시간(계산)
0,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200117193331,20200117201417,00:40:00,...,31,2020-01-17 19:33:31,2020,1,17,20,14,17,2020-01-17 20:14:17,0 days 00:40:46
1,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200130143545,20200130150645,00:30:31,...,45,2020-01-30 14:35:45,2020,1,30,15,6,45,2020-01-30 15:06:45,0 days 00:31:00
2,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200105005728,20200105011329,00:15:35,...,28,2020-01-05 00:57:28,2020,1,5,1,13,29,2020-01-05 01:13:29,0 days 00:16:01
3,내린천휴게소 양양방향,3,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),100.0,DC콤보,20200120120215,20200120124708,00:41:00,...,15,2020-01-20 12:02:15,2020,1,20,12,47,8,2020-01-20 12:47:08,0 days 00:44:53
4,한국광기술원,1,광주광역시,북구,광주광역시 북구 첨단 벤처로 108번길 9,50.0,DC차데모+AC3상+DC콤보,20200123085417,20200123092554,00:31:14,...,17,2020-01-23 08:54:17,2020,1,23,9,25,54,2020-01-23 09:25:54,0 days 00:31:37


In [217]:
df.iloc[:5, :12]

,충전소명,충전기ID,지역,시군구,주소,충전기용량(kW),충전기타입,충전시작일시,충전종료일시,충전시간,충전량,충전용량
0,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200117193331,20200117201417,00:40:00,25.47,50.0
1,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200130143545,20200130150645,00:30:31,17.57,50.0
2,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),50.0,DC차데모+AC3상+DC콤보,20200105005728,20200105011329,00:15:35,6.98,50.0
3,내린천휴게소 양양방향,3,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),100.0,DC콤보,20200120120215,20200120124708,00:41:00,15.30,100.0
4,한국광기술원,1,광주광역시,북구,광주광역시 북구 첨단 벤처로 108번길 9,50.0,DC차데모+AC3상+DC콤보,20200123085417,20200123092554,00:31:14,21.92,50.0


In [218]:
df.to_csv('2020-22년_급속충전_요일분리.csv', encoding='utf-8')

In [219]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11665822 entries, 0 to 11665821
Data columns (total 27 columns):
 #   Column      Dtype          
---  ------      -----          
 0   충전소명        object         
 1   충전기ID       int64          
 2   지역          object         
 3   시군구         object         
 4   주소          object         
 5   충전기용량(kW)   float64        
 6   충전기타입       object         
 7   충전시작일시      int64          
 8   충전종료일시      object         
 9   충전시간        object         
 10  충전량         float64        
 11  충전용량        object         
 12  충전시작_년도     int32          
 13  충전시작_월      int32          
 14  충전시작_일      int32          
 15  충전시작_시      int32          
 16  충전시작_분      int32          
 17  충전시작_초      int32          
 18  충전시작일시(변환)  datetime64[ns] 
 19  충전종료_년도     int32          
 20  충전종료_월      int32          
 21  충전종료_일      int32          
 22  충전종료_시      int32          
 23  충전종료_분      int32          
 24  충전종료_초      int32     

### 필요없는 열 제거

In [223]:
mod_df = df.drop(['충전기용량(kW)', '충전시작_년도', '충전시작_월', '충전시작_일', '충전시작_시', '충전시작_월', '충전시작_일', '충전시작_시', '충전시작_분', '충전시작_초', 
         '충전종료_년도', '충전종료_월', '충전종료_일', '충전종료_시', '충전종료_월', '충전종료_일', '충전종료_시', '충전종료_분', '충전종료_초'], axis=1)
mod_df.head()

,충전소명,충전기ID,지역,시군구,주소,충전기타입,충전시작일시,충전종료일시,충전시간,충전량,충전용량,충전시작일시(변환),충전종료일시(변환),충전시간(계산)
0,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),DC차데모+AC3상+DC콤보,20200117193331,20200117201417,00:40:00,25.47,50.0,2020-01-17 19:33:31,2020-01-17 20:14:17,0 days 00:40:46
1,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),DC차데모+AC3상+DC콤보,20200130143545,20200130150645,00:30:31,17.57,50.0,2020-01-30 14:35:45,2020-01-30 15:06:45,0 days 00:31:00
2,내린천휴게소 양양방향,1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),DC차데모+AC3상+DC콤보,20200105005728,20200105011329,00:15:35,6.98,50.0,2020-01-05 00:57:28,2020-01-05 01:13:29,0 days 00:16:01
3,내린천휴게소 양양방향,3,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향),DC콤보,20200120120215,20200120124708,00:41:00,15.30,100.0,2020-01-20 12:02:15,2020-01-20 12:47:08,0 days 00:44:53
4,한국광기술원,1,광주광역시,북구,광주광역시 북구 첨단 벤처로 108번길 9,DC차데모+AC3상+DC콤보,20200123085417,20200123092554,00:31:14,21.92,50.0,2020-01-23 08:54:17,2020-01-23 09:25:54,0 days 00:31:37


In [232]:
mod_df.to_csv('2020-22년_급속충전_일시정리.csv')

In [234]:
mod_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11665822 entries, 0 to 11665821
Data columns (total 14 columns):
 #   Column      Dtype          
---  ------      -----          
 0   충전소명        object         
 1   충전기ID       int64          
 2   지역          object         
 3   시군구         object         
 4   주소          object         
 5   충전기타입       object         
 6   충전시작일시      int64          
 7   충전종료일시      object         
 8   충전시간        object         
 9   충전량         float64        
 10  충전용량        object         
 11  충전시작일시(변환)  datetime64[ns] 
 12  충전종료일시(변환)  datetime64[ns] 
 13  충전시간(계산)    timedelta64[ns]
dtypes: datetime64[ns](2), float64(1), int64(2), object(8), timedelta64[ns](1)
memory usage: 1.2+ GB


### raw 충전시간과 계산값 비교

In [227]:
df['충전시간'].head()

0    00:40:00
1    00:30:31
2    00:15:35
3    00:41:00
4    00:31:14
Name: 충전시간, dtype: object

In [224]:
df['충전시간'].dtype

dtype('O')

In [226]:
(df['충전시간'] != df['충전시간(계산)']).sum()

11665822

In [230]:
temp = pd.to_timedelta(df['충전시간'], errors='ignore')
temp.head()

0    00:40:00
1    00:30:31
2    00:15:35
3    00:41:00
4    00:31:14
Name: 충전시간, dtype: object

In [231]:
(temp != df['충전시간(계산)']).sum()

11665822

In [233]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11665822 entries, 0 to 11665821
Data columns (total 27 columns):
 #   Column      Dtype          
---  ------      -----          
 0   충전소명        object         
 1   충전기ID       int64          
 2   지역          object         
 3   시군구         object         
 4   주소          object         
 5   충전기용량(kW)   float64        
 6   충전기타입       object         
 7   충전시작일시      int64          
 8   충전종료일시      object         
 9   충전시간        object         
 10  충전량         float64        
 11  충전용량        object         
 12  충전시작_년도     int32          
 13  충전시작_월      int32          
 14  충전시작_일      int32          
 15  충전시작_시      int32          
 16  충전시작_분      int32          
 17  충전시작_초      int32          
 18  충전시작일시(변환)  datetime64[ns] 
 19  충전종료_년도     int32          
 20  충전종료_월      int32          
 21  충전종료_일      int32          
 22  충전종료_시      int32          
 23  충전종료_분      int32          
 24  충전종료_초      int32     